In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [113]:
df = pd.read_csv('/Users/siboraseranaj/Desktop/project/STA440- Project/IST_corrected1.csv')

/Users/siboraseranaj/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [114]:
# result at six months
df["OCCODE"].unique()

array([2, 4, 3, 9, 1, 0])

## Create the dataset for both objectives

In [115]:
# stroke columns 'DRSISC', "DRSH", "DRSUNK", "DPE"

columns = ['RDELAY', 'SEX','AGE','RCONSC','RVISINF','RSBP','RDEF1',
           'RDEF2','RDEF3','RDEF4','RDEF5','RDEF6','RDEF7',
           'RDEF8','STYPE','RXASP','RATRIAL','RXHEP','TD','OCCODE','DRSISC', "DRSH", "DRSUNK", "DPE",'DRSISCD', "DRSHD", "DRSUNKD"]

In [116]:
obj1_df = df[columns]

In [117]:
def change_OCCODE(row):
    if row == 1:
        return 1
    elif row == 9 or row == 0:
        return None
    else:
        return 0

obj1_df['OCCODE'] = obj1_df['OCCODE'].apply(lambda x: change_OCCODE(x))

<ipython-input-117-7b95df005131>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obj1_df['OCCODE'] = obj1_df['OCCODE'].apply(lambda x: change_OCCODE(x))


In [118]:
obj1_df = obj1_df.dropna(subset=['OCCODE']).reset_index(drop = True)

In [119]:
def stroke_func(row):
    if row[0] == 'Y' or row[1] == 'Y' or row[2] == 'Y':
        return 1
    else:
        return 0
    
obj1_df['recurrentstroke'] = obj1_df[['DRSISC', "DRSH", "DRSUNK"]].apply(lambda x: stroke_func(x),axis = 1)

In [120]:
#def change_DF(row):
 #   if row[0] == 1 and (row[1]) > 185:
  #      return str(row[1])+'+'
  #  else:
   #     return str(row[1])
    
#obj1_df['TD'] = obj1_df[['OCCODE','TD']].apply(lambda x: change_DF(x),axis = 1)



In [121]:
obj1_df['DRSISCD'].fillna('None',inplace = True)
obj1_df['DRSHD'].fillna('None',inplace = True)
obj1_df['DRSUNKD'].fillna('None',inplace = True)

In [122]:
def recurrent_dates(row):
    if row[0] == 'None' and row[1] == 'None' and row[2] == 'None':
        return 14
    elif row[0] != 'None' and row[1] == 'None' and row[2] == 'None':
        return row[0]
    elif row[0] == 'None' and row[1] != 'None' and row[2] == 'None':
        return row[1]
    elif row[0] == 'None' and row[1] == 'None' and row[2] != 'None':
        return row[2]
    
obj1_df['recurrentstrokedate'] = obj1_df[['DRSISCD', "DRSHD", "DRSUNKD"]].apply(lambda x: recurrent_dates(x), axis = 1)
    
    

In [123]:
def change_OCD(row):
    if row[0] == 1 and row[1] > 183:
        return 0
    else:
        return row[0]
    
obj1_df['OCCODE'] = obj1_df[['OCCODE','TD']].apply(lambda x:change_OCD(x), axis = 1)

In [124]:
def change_TD(row):
    if row > 183:
        return 183
    else:
        return row
    
obj1_df['TD'] = obj1_df['TD'].apply(lambda x:change_TD(x))

In [125]:
obj1_df[['OCCODE','TD']].head(5)

,OCCODE,TD
0,0.0,183.0
1,0.0,183.0
2,0.0,183.0
3,0.0,183.0
4,0.0,183.0
5,0.0,183.0
6,0.0,183.0
7,0.0,183.0
8,0.0,183.0
9,0.0,183.0


In [126]:
def treatment(row):
    if row[0] == 'Y' and row[1] == 'N':
        return 'ASP'
    elif row[0] == 'N' and (row[1] == 'H' or row[1] == 'L' or row[1] == 'M'):
        return 'HEP'
    elif row[0] == 'Y' and (row[1] == 'H' or row[1] == 'L' or row[1] == 'M'):
        return 'BOTH'
    else:
        return 'NONE'

obj1_df['trt'] = obj1_df[['RXASP','RXHEP']].apply(lambda x: treatment(x), axis = 1)    

In [127]:
# only people with aspirin or heparin 9146 patients
HEPorASP = obj1_df.loc[((obj1_df["trt"] == "ASP") | (obj1_df["trt"] == "HEP"))]

In [128]:
HEPorASP = HEPorASP.dropna(subset=['RATRIAL']).reset_index(drop = True)

In [129]:
len(HEPorASP)

9146

In [130]:
# numerical value for amount of symptoms presented at admission
def count_def(row):
    count = 0
    for i in range(len(row)):
        if row[i] == 'Y':
            count += 1
    return count

HEPorASP['physicaldeficit'] = HEPorASP[['RDEF1', 'RDEF2',
       'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8']].apply(lambda x: count_def(x),axis = 1)

In [131]:
print(HEPorASP.columns)
obj_1_1 = HEPorASP.copy()

Index(['RDELAY', 'SEX', 'AGE', 'RCONSC', 'RVISINF', 'RSBP', 'RDEF1', 'RDEF2',
       'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'STYPE', 'RXASP',
       'RATRIAL', 'RXHEP', 'TD', 'OCCODE', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE',
       'DRSISCD', 'DRSHD', 'DRSUNKD', 'recurrentstroke', 'recurrentstrokedate',
       'trt', 'physicaldeficit'],
      dtype='object')


In [132]:
HEPorASP = HEPorASP.rename(columns={'RVISINF':"infarct",
                       'RSBP':"bloodpressure" , "OCCODE": "death", "SEX":"sex", "AGE":"age",
                       'STYPE':"strokesubtype", 'TD':"timedeath",
                       'RDELAY':"timedelay",'RCONSC':'concious', 'RATRIAL' : 'atrialfibrillation'})


In [133]:
HEPorASP.drop(['RDEF1','RDEF2','RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'RXASP', "RXHEP", 'DRSISC', 'DRSH', 'DRSUNK', 'DPE', 'DRSISCD', 'DRSHD', 'DRSUNKD'], inplace=True, axis=1)

In [134]:
def change_recurrentstroke(row):
    if row[0] == 1 and row[1] > 14:
        return 0
    else:
        return row[0]
    
HEPorASP['recurrentstroke'] = HEPorASP[['recurrentstroke','recurrentstrokedate']].apply(lambda x:change_recurrentstroke(x), axis = 1)

In [135]:
def change_recurrentstrokedate(row):
    if row > 14:
        return 14
    else:
        return row
    
HEPorASP['recurrentstrokedate'] = HEPorASP['recurrentstrokedate'].apply(lambda x:change_recurrentstrokedate(x))

In [136]:
HEPorASP["recurrentstrokedate"].unique()

array([14.,  1.,  5.,  4.,  0., 13.,  2.,  9.,  7., 10.,  6.,  3., 12.,
        8., 11., nan])

In [137]:
HEPorASP.to_csv('/Users/siboraseranaj/Desktop/project/STA440- Project/death.csv')

# not used

In [97]:
#obj_1_1["RXHEP"].unique()

In [98]:
#obj_1_1 = obj_1_1[obj_1_1["RXASP"] == "N"].reset_index(drop = True)

In [99]:
#obj_1_1 = obj_1_1[obj_1_1["RXHEP"] != "N"].reset_index(drop = True)

In [100]:
#def trthep(row):
  #if row == "L":
        #return "L"
    #else:
        #return "H"
    
#obj_1_1["RXHEP"] = obj_1_1["RXHEP"].apply(lambda x : trthep(x))

In [101]:
#obj_1_1 = obj_1_1.dropna(subset=['RATRIAL']).reset_index(drop = True)

In [102]:
#obj_1_1['physicaldeficit'] = obj_1_1[['RDEF1', 'RDEF2',
       #'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8']].apply(lambda x: count_def(x),axis = 1)
#obj_1_1.head()

IndentationError: unexpected indent (<ipython-input-102-f6ac8931ee1a>, line 2)

In [103]:
#obj_1_1 = obj1_df.loc[((obj1_df["trt"] == "ASP") | (obj1_df["trt"] == "HEP"))]
#len(obj_1_1)
#obj_1_1["trt"].unique()

In [104]:
#print(obj_1_1.columns)

Index(['RDELAY', 'SEX', 'AGE', 'RCONSC', 'RVISINF', 'RSBP', 'RDEF1', 'RDEF2',
       'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'STYPE', 'RXASP',
       'RATRIAL', 'RXHEP', 'TD', 'OCCODE', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE',
       'DRSISCD', 'DRSHD', 'DRSUNKD', 'recurrentstroke', 'recurrentstrokedate',
       'trt'],
      dtype='object')


In [105]:
#obj_1_1 = obj_1_1.rename(columns={'RVISINF':"infarct",
                       #'RSBP':"bloodpressure" , "OCCODE": "death", "SEX":"sex", "AGE":"age",
                       #'STYPE':"strokesubtype", 'TD':"timedeath",
                       #'RDELAY':"timedelay",'RCONSC':'concious', 'RATRIAL' : 'atrialfibrillation'})


In [106]:
#obj_1_1.drop(['RDEF1','RDEF2','RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'DRSISC', 'DRSH', 'DRSUNK','DRSISCD', 'DRSHD','DRSUNKD',"RXASP"], inplace=True, axis=1)

In [107]:
#def change_recurrentstroke(row):
    #if row[0] == 1 and row[1] > 14:
        #return 0
    #else:
        #return row[0]
    
#obj1_df['recurrentstroke'] = obj1_df[['recurrentstroke','recurrentstrokedate']].apply(lambda x:change_recurrentstroke(x), axis = 1)

In [108]:
#obj_1_1.to_csv('/Users/siboraseranaj/Desktop/project/STA440- Project/HEP_LorH.csv')

In [111]:
#obj1_df

,RDELAY,SEX,AGE,RCONSC,RVISINF,RSBP,RDEF1,RDEF2,RDEF3,RDEF4,...,DRSISC,DRSH,DRSUNK,DPE,DRSISCD,DRSHD,DRSUNKD,recurrentstroke,recurrentstrokedate,trt
0,17,M,69,D,Y,140,N,N,N,Y,...,N,N,NaN,N,None,None,None,0.0,14.0,ASP
1,10,M,76,F,N,150,Y,Y,Y,N,...,N,N,NaN,N,None,None,None,0.0,14.0,HEP
2,43,F,71,F,N,170,Y,Y,Y,N,...,N,N,NaN,N,None,None,None,0.0,14.0,ASP
3,6,M,81,F,N,170,N,N,N,Y,...,N,N,NaN,N,None,None,None,0.0,14.0,HEP
4,20,M,78,F,N,170,Y,Y,Y,N,...,N,N,NaN,N,None,None,None,0.0,14.0,BOTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19280,42,M,66,F,Y,120,Y,Y,Y,Y,...,N,N,N,N,None,None,None,0.0,14.0,HEP
19281,47,M,75,F,Y,210,Y,Y,Y,N,...,N,N,N,N,None,None,None,0.0,14.0,HEP
19282,21,M,77,F,N,220,Y,Y,Y,N,...,Y,N,N,N,1,None,None,1.0,1.0,NONE
19283,40,F,87,D,Y,160,N,Y,Y,Y,...,N,N,N,N,None,None,None,0.0,14.0,NONE


In [109]:
#def change_recurrentstrokedate(row):
    #if row > 14:
        #return 14
   # else:
        #return row
    
#obj1_df['recurrentstrokedate'] = obj1_df['recurrentstrokedate'].apply(lambda x:change_recurrentstrokedate(x))

In [110]:
#obj1_df["recurrentstrokedate"].unique()

array([14.,  2.,  1.,  0.,  3.,  4., 12.,  7.,  5.,  9., 10., 11., 13.,
        8., nan,  6.])